# Ref: MyAppollo [LangChain 怎麼玩？為了荷包著想，管好你的 prompt 長度(size)](https://myapollo.com.tw/blog/langchain-managing-prompt-size/#google_vignette)

## 這個例子相較於LangChain用

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")

number_of_tokens = llm.get_num_tokens_from_messages([HumanMessage("Hi, there. How are you today?")])
print(f"Tokens: {number_of_tokens}")

num = llm.get_num_tokens('Hi, there. How are you today?')
print(f'Tokens: {num}')


Tokens: 16
Tokens: 9


在把prompt傳給LLM之前，修改prompt的長度
```
prompt | modify promtp | LLM
```
prompt template 在加入 input value 之後會變成`ChatPromptValue`的實例(instance)，該實例可以呼叫`.to_messages()`，再丟回`.get_num_tokens_from_messages`得到tokens的總數。超過預計長度時，我們可以把舊的messages丟掉，再把剩下的messages用ChatPromptValue回傳

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.messages import HumanMessage, AIMessage


chat_history = []

# >>> original code, incorrect >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# def condense_prompt(prompt: ChatPromptValue) -> ChatPromptValue:
#     messages = prompt.to_messages()
#     print(f"===========================\n {messages}\n===========================")
#     num_tokens = llm.get_num_tokens_from_messages(messages)
#     recent_messages = messages[2:]
#     while num_tokens > 50:
#         recent_messages = recent_messages[2:] # remove 2 messages。每一輪對話會有一次Human，一次AI
#         num_tokens = llm.get_num_tokens_from_messages(
#             messages[:2] + recent_messages
#         )
#     # also update chat history
#     chat_history = recent_messages
#     messages = messages[:2] + recent_messages
#     return ChatPromptValue(messages=messages)
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# >>> Sean modified code >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def condense_prompt(prompt: ChatPromptValue) -> ChatPromptValue:
    messages = prompt.to_messages()
    # print(f"===========================\n {messages}\n===========================")
    num_tokens = llm.get_num_tokens_from_messages(messages)
    recent_messages = messages[1:]
    # while ( num_tokens > 100 and len(recent_messages) > 4 ):
    while num_tokens > 50:
        # remove 2 messages。每一輪對話會有一次Human，一次AI. 但是我們開頭有system message
        sys_message = messages[0]
        recent_messages = messages[3:]
        messages = [sys_message] + recent_messages
        # print(f"===========================\n {messages}\n===========================")
        num_tokens = llm.get_num_tokens_from_messages(
            [sys_message] + recent_messages
        )
    # also update chat history
    chat_history = recent_messages
    messages = [messages[0]] + recent_messages
    return ChatPromptValue(messages=messages)
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a powerful chat bot.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
])

chain = prompt | condense_prompt | llm

input_text = input('>>> ')
while input_text.lower() != 'bye':
    if input_text:
        print(f"===========================\n {chat_history}\n===========================")
        response = chain.invoke({
            'input': input_text,
            'chat_history': chat_history,
        })
        chat_history.append(HumanMessage(content=input_text))
        chat_history.append(AIMessage(content=response.content))
        # print(f"===========================\n {chat_history}\n===========================")
        print(response.content)
    input_text = input('>>> ')

 []
The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it collides with molecules and small particles in the air. Sunlight is composed of many colors, each with different wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.

As sunlight passes through the atmosphere, the shorter wavelengths (blue and violet) are scattered in all directions more than the longer wavelengths (red and yellow). Although violet light is scattered even more than blue, our eyes are more sensitive to blue light, and some of the violet light is absorbed by the ozone layer. As a result, we perceive the sky as blue during the day.

At sunrise and sunset, the sun is lower on the horizon, and its light has to pass through a greater thickness of the atmosphere. This causes even more scattering of the shorter wavelengths, allowing the longer wavelengths (reds and oranges) to dominate, which is why we see beautiful w